# Random Forest

Случайный лес — модель, состоящая из множества деревьев решений. Вместо того,чтобы просто усреднять прогнозы разных деревьев (такая концепция называется просто «лес»), эта модель использует две ключевые концепции, которые и делают этот лес случайным.

**1.** Случайная выборка образцов из набора данных при построении деревьев.

**2.** При разделении узлов выбираются случайные наборы параметров.

**Все деревья строятся независимо по следующей схеме:**

**1.** Выбирается подвыборка обучающей выборки, по ней строится дерево (для каждого дерева — своя подвыборка).

**2.** Для построения каждого расщепления в дереве просматриваем max_features случайных признаков (для каждого нового расщепления — свои случайные признаки).

**3.** Выбираем наилучшие признак и расщепление по нему (по заранее заданному критерию). Дерево строится, как правило, до исчерпания выборки (пока в листьях не останутся представители только одного класса), но в современных реализациях есть параметры, которые ограничивают высоту дерева, число объектов в листьях и число объектов в подвыборке, при котором проводится расщепление.

В задаче регрессии ответы деревьев усредняются, в задаче классификации принимается решение голосованием по большинству. 

### А зачем модель нужна, когда есть уже прекрасные деревья

Алгоритм дерева решений переобучается, если не ограничить его максимальную глубину. Он обладает неограниченной гибкостью и может разрастаться, пока не достигнет состояния идеальной классификации, в которой каждому образцу из набора данных будет соответствовать один лист. Если вернуться назад к созданию дерева и ограничить его глубину двумя слоями (сделав только одно разделение), классификация больше не будет на 100 % верной. Мы уменьшаем вариативность за счёт увеличения погрешности.

В качестве альтернативы ограничению глубины, которое ведёт к уменьшению вариативности (хорошо) и увеличению погрешности (плохо), мы можем собрать множество деревьев в единую модель. Это и будет классификатор на основе комитета деревьев принятия решений или просто «случайный лес».

## Основные параметры модели

### Число деревьев — n_estimators

Чем больше деревьев, тем лучше качество, но время настройки и работы RF также пропорционально увеличиваются. Обратите внимание, что часто при увеличении n_estimators качество на обучающей выборке повышается (может даже доходить до 100%), а качество на тесте выходит на асимптоту (можно прикинуть, скольких деревьев Вам достаточно).


![](https://alexanderdyakonov.files.wordpress.com/2016/11/n_estimators.png?w=768)

### Максимальная глубина деревьев — max_depth
Ясно, что чем меньше глубина, тем быстрее строится и работает RF. При увеличении глубины резко возрастает качество на обучении, но и на контроле оно, как правило, увеличивается. Рекомендуется использовать максимальную глубину (кроме случаев, когда объектов слишком много и получаются очень глубокие деревья, построение которых занимает значительное время). При использовании неглубоких деревьев изменение параметров, связанных с ограничением числа объектов в листе и для деления, не приводит к значимому эффекту (листья и так получаются «большими»). Неглубокие деревья рекомендуют использовать в задачах с большим числом шумовых объектов (выбросов).

![](https://alexanderdyakonov.files.wordpress.com/2016/11/max_depth.png?w=768)

### Число признаков для выбора расщепления — max_features
График качества на тесте от значения этого праметра унимодальный, на обучении он строго возрастает. При увеличении max_features увеличивается время построения леса, а деревья становятся «более однообразными». По умолчанию он равен sqrt(n) в задачах классификации и n/3 в задачах регрессии.

## Проверим алгоритм на деле


In [12]:
import pandas as pd
import numpy as np
df = pd.read_csv('2015.csv').sample(100000, random_state = 53)
df

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,...,_PAREC1,_PASTAE1,_LMTACT1,_LMTWRK1,_LMTSCL1,_RFSEAT2,_RFSEAT3,_FLSHOT6,_PNEUMO2,_AIDTST3
261167,34.0,7.0,b'07282015',b'07',b'28',b'2015',1100.0,2.015005e+09,2.015005e+09,1.0,...,2.0,2.0,3.0,3.0,4.0,1.0,1.0,NaN,NaN,9.0
293775,38.0,1.0,b'01082015',b'01',b'08',b'2015',1100.0,2.015000e+09,2.015000e+09,1.0,...,9.0,9.0,2.0,2.0,3.0,1.0,1.0,2.0,2.0,2.0
386309,50.0,1.0,b'01152015',b'01',b'15',b'2015',1100.0,2.015001e+09,2.015001e+09,1.0,...,1.0,1.0,3.0,3.0,4.0,1.0,1.0,1.0,1.0,1.0
274432,36.0,8.0,b'09142015',b'09',b'14',b'2015',1200.0,2.015000e+09,2.015000e+09,NaN,...,1.0,1.0,3.0,3.0,4.0,1.0,1.0,NaN,NaN,2.0
217891,29.0,7.0,b'07152015',b'07',b'15',b'2015',1200.0,2.015003e+09,2.015003e+09,1.0,...,9.0,9.0,3.0,3.0,4.0,9.0,9.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186786,26.0,1.0,b'02222015',b'02',b'22',b'2015',1100.0,2.015003e+09,2.015003e+09,1.0,...,4.0,2.0,2.0,2.0,3.0,1.0,1.0,NaN,NaN,2.0
338490,45.0,9.0,b'09212015',b'09',b'21',b'2015',1100.0,2.015004e+09,2.015004e+09,1.0,...,4.0,2.0,3.0,3.0,4.0,1.0,1.0,NaN,NaN,2.0
11078,2.0,6.0,b'06102015',b'06',b'10',b'2015',1100.0,2.015003e+09,2.015003e+09,NaN,...,4.0,2.0,3.0,3.0,4.0,1.0,1.0,NaN,NaN,2.0
178493,25.0,11.0,b'11112015',b'11',b'11',b'2015',1200.0,2.015004e+09,2.015004e+09,1.0,...,4.0,2.0,2.0,2.0,3.0,1.0,1.0,1.0,1.0,2.0


In [13]:
df = df.select_dtypes('number')

In [14]:
df['_RFHLTH'] = df['_RFHLTH'].replace({2: 0})
df = df.loc[df['_RFHLTH'].isin([0, 1])].copy()
df = df.rename(columns = {'_RFHLTH': 'label'})
df['label'].value_counts()

D:\Python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


1.0    80984
0.0    18721
Name: label, dtype: int64

In [15]:
df = df.drop(columns = ['POORHLTH', 'PHYSHLTH', 'GENHLTH', 'PAINACT2', 
                        'QLMENTL2', 'QLSTRES2', 'QLHLTH2', 'HLTHPLN1', 'MENTHLTH'])

In [16]:
from sklearn.model_selection import train_test_split

labels = np.array(df.pop('label'))
train, test, train_labels, test_labels = train_test_split(df, labels, 
                                                          stratify = labels,
                                                          test_size = 0.3, 
                                                          random_state = 53)

In [17]:
train = train.fillna(train.mean())
test = test.fillna(test.mean())

features = list(train.columns)

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve, accuracy_score
tree = DecisionTreeClassifier(random_state=53)
tree.fit(train, train_labels)
predictions = tree.predict(test)

print(f'accuracy: {accuracy_score(test_labels, predictions)}')

accuracy: 0.7606980476063119


In [21]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, 
                               random_state=53, 
                               max_features = 'sqrt',
                               n_jobs=-1, verbose = 1)

model.fit(train, train_labels)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   15.0s finished


RandomForestClassifier(max_features='sqrt', n_jobs=-1, random_state=53,
                       verbose=1)

In [22]:
train_rf_predictions = model.predict(train)
train_rf_probs = model.predict_proba(train)[:, 1]

rf_predictions = model.predict(test)
rf_probs = model.predict_proba(test)[:, 1]

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.2s finished


In [28]:
print(f'accuracy: {accuracy_score(test_labels, rf_predictions)}')

accuracy: 0.8642016581973789
